In [1]:
!pip install ipyleaflet rdflib requests geodaisy pyproj shapely geopandas geomet tqdm

# Plot G-NAF addresses for a given ASGS MeshBlock

This notebook demonstrates how you use Loc-I to identify GNAF Addresses from an input ASGS MeshBlock.
This notebook shows you how to traverse across from one geographic dataset to another via `Loc-I Linksets`, specifically ASGS->GNAF.
We show how to achieve this using the Loc-I Integration API (https://api.loci.cat).

In [2]:
import urllib, json, requests

In [12]:
# input meshblock
meshblock_uri ='http://linked.data.gov.au/dataset/asgs2016/meshblock/80000060000'
meshblock_uri = 'http://linked.data.gov.au/dataset/asgs2016/meshblock/20444240000'
meshblock_uri = 'http://linked.data.gov.au/dataset/asgs2016/meshblock/20426000000'
#tell me more about the meshblock
# get semantic info about the feature
header={'Accept': 'text/turtle'}
url = meshblock_uri
r = requests.get(url, headers=header)
res_body = r.text
print(res_body)

@prefix asgs: <http://linked.data.gov.au/def/asgs#> .
@prefix asgs-cat: <http://linked.data.gov.au/def/asgs-cat/> .
@prefix asgs-id: <http://linked.data.gov.au/def/asgs/id#> .
@prefix data: <http://linked.data.gov.au/def/datatype/> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .
@prefix geox: <http://linked.data.gov.au/def/geox#> .
@prefix loci: <http://linked.data.gov.au/def/loci#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://linked.data.gov.au/dataset/asgs2016/destinationzone/213202172> a asgs:DestinationZone ;
    dcterms:identifier "213202172"^^asgs-id:dznCode2016 ;
    geo:sfContains <http://linked.data.gov.au/dataset/asgs2016/meshblock/20426000000> .

<http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/24970> a asgs:LocalGovernm

In [13]:
#you could use rdflib to do some more digging into the semantic description to get the geometry... 
import rdflib
g=rdflib.Graph()
g.load(meshblock_uri)

#for s,p,o in g:
#    print(s, p, o)

geom_uri = None
for row in g.query(
            '''
            PREFIX geo: <http://www.opengis.net/ont/geosparql#>
            select ?g where { ?feature geo:hasGeometry ?g .}
            '''):
        geom_uri = (row.g)
if(geom_uri != None):
    print(geom_uri)

http://gds.loci.cat/geometry/asgs16_mb/20426000000


In [14]:
# get the geojson representation of the geom
header={'Accept': 'application/json'}
r = requests.get(geom_uri, headers=header)
geojson_data = r.json()

params={'_view': 'centroid'}
r = requests.get(geom_uri, headers=header, params=params)
centroid_geojson_data = r.json()

In [15]:
#draw the geometry for the feature on a map
import ipyleaflet as ipy 
import ipywidgets as ipyw
from ipyleaflet import GeoJSON, Map, Marker

x_coord = centroid_geojson_data['coordinates'][1]
y_coord = centroid_geojson_data['coordinates'][0]

map = ipy.Map(center=[x_coord, y_coord], zoom=15)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json1 = GeoJSON(data=geojson_data, style = {'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.5})
map.add_layer(geo_json1)
map

Map(center=[-37.910620529, 145.134765925], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in…

## Find the related G-NAF addresses via the Loc-I Integration API

We want to traverse the Loc-I linkset to get the G-NAF addresses

In [16]:
#we give the feature_type URI to the Loc-I API
feature_type = 'http://linked.data.gov.au/def/gnaf#Address'

payload = {
    'uri': meshblock_uri, 
    'areas': 'false',
    'proportion': 'false',
    'contains': 'false',
    'within':'true',
    'crosswalk':'true',
    'output_type' : feature_type,
    'count': 100,
    'offset': 0
}

url = "https://api.loci.cat/api/v1/location/overlaps"
r = requests.get(url, params=payload)
res = r.json()
res

{'meta': {'count': 156, 'offset': 0},
 'overlaps': [{'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAVIC413712007'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAVIC419752607'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAVIC419752608'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAVIC419778984'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAVIC420047980'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAVIC420047982'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAVIC420338218'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAVIC420645459'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAVIC420947680'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAVIC420964922'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAVIC421086193'},
  {'uri': 'http://linked.d

Let's go ahead and process each address to get the spatial object and add them to the map

In [17]:
#load the resources into a graph object
address_graph = rdflib.Graph()
from tqdm import tqdm

for r in tqdm(res['overlaps']):
    address_graph.load(r['uri'])

#for s,p,o in address_graph:
#    print(s, p, o)

100%|██████████| 156/156 [01:01<00:00,  2.55it/s]


In [23]:
list_of_geom_obj = []
for row in address_graph.query(
            '''
            PREFIX geo: <http://www.opengis.net/ont/geosparql#>
            select distinct ?wkt ?type ?label ?note
            where { 
                   ?f rdfs:comment ?note .
                   ?f geo:hasGeometry ?g .
                   ?g geo:asWKT ?wkt .
                   ?g dct:type ?type .
                   ?g rdfs:label ?label .
               }
            ORDER BY ?note
            '''):
        geom = (str(row.wkt), str(row.type), row.label.value, row.note.value)
        list_of_geom_obj.append(geom)
        print(geom)

('<http://www.opengis.net/def/crs/EPSG/0/4283> POINT(145.13518899 -37.90589628)', 'http://gnafld.net/def/gnaf/code/GeocodeTypes#StreetLocality', 'Street Locality', '100 Normanby Road, Clayton, VIC 3168')
('<http://www.opengis.net/def/crs/EPSG/0/4283> POINT(145.13390076 -37.91515521)', 'http://gnafld.net/def/gnaf/code/GeocodeTypes#FrontageCentreSetback', 'Frontage Centre Setback', '13 College Walk, Clayton, VIC 3168')
('<http://www.opengis.net/def/crs/EPSG/0/4283> POINT(145.13885566 -37.91597185)', 'http://gnafld.net/def/gnaf/code/GeocodeTypes#StreetLocality', 'Street Locality', '136 Wellington Road, Clayton, VIC 3168')
('<http://www.opengis.net/def/crs/EPSG/0/4283> POINT(145.13390076 -37.91515521)', 'http://gnafld.net/def/gnaf/code/GeocodeTypes#FrontageCentreSetback', 'Frontage Centre Setback', '15 Innovation Walk, Clayton, VIC 3168')
('<http://www.opengis.net/def/crs/EPSG/0/4283> POINT(145.13390076 -37.91515521)', 'http://gnafld.net/def/gnaf/code/GeocodeTypes#FrontageCentreSetback', '

In [24]:
#setup a new leaflet map with the meshblock
map2 = ipy.Map(center=[x_coord, y_coord], zoom=14)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))
map2.add_layer(geo_json1)

import re
from pyproj import Transformer
from shapely.wkt import dumps, loads
from geomet import wkt
from shapely.ops import transform

#add layer for each wkt
for geom_obj in list_of_geom_obj:
    #print(geom_obj[0])
    m = re.search('<(.*)> (.*)', geom_obj[0])
    crs = m.group(1)
    wktdata = m.group(2)
    #transform crs to epsg 4326 if needed...
    if crs == 'http://www.opengis.net/def/crs/EPSG/0/4283':
        project = Transformer.from_crs("EPSG:4283", "EPSG:4326")            
        wktdata = loads(wktdata)
        g2 = transform(project.transform, wktdata)  # apply projection   
        wktdata = dumps(g2)
        #print(wktdata)
    geojson_data = wkt.loads(wktdata)
    #print(geojson_data)
            
    geojson = GeoJSON(data=geojson_data, style = {'color': 'blue', 'fillcolor': 'blue', 'opacity':1, 'weight':1.9, 'fillOpacity':0.1})
    map2.add_layer(geojson)
map2

Map(center=[-37.910620529, 145.134765925], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in…

Ta-da!

You can see GNAF addresses as blue markers within the meshblock (shown in red). 

We hope you enjoyed that little demonstration of how to traverse from a ASGS MeshBlock across to the GNAF Address based on linkset relationships registered in Loc-I.